<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH17_03_%EA%B5%AC%EA%B8%80_BERT%EC%9D%98_%EB%A7%88%EC%8A%A4%ED%81%AC%EB%93%9C_%EC%96%B8%EC%96%B4%EB%AA%A8%EB%8D%B8(Masked_Language_Model)_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.7 MB/s eta 0:00:00


### 1. 마스크드 언어 모델과 토크나이저
-------------------
transformers 패키지를 이용하여 모델과 토크나이저를 로드함. BERT는 이미 누군가가 학습해 둔 모델을 사용하는 것이므로, 우리가 사용하는 모델과 토크나이저가 항상 맵핑 관계에 있어야 함. 

예를 들어, A라는 이름의 BERT를 사용하는데, B라는 이름의 BERT의 토크나이저를 사용하면 모델은 텍스트를 제대로 이해할 수 없음. A라는 BERT의 토크나이저는 '사과'라는 단어를 36번으로 정수 인코딩하는 반면에, B라는 BERT의 토크나이저는 '사과'라는 단어를 42번으로 정수 인코딩하는 등 다르기 때문임 

In [2]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

 TFBertForMaskedLM.from_pretrained('모델이름')을 넣으면 [MASK]라고 되어 있는 단어를 맞추기 위한 마스크드 언어 모델링을 위한 구조로 BERT를 로드함. 즉, BERT를 마스크드 언어모델 형태로 로드함


AutoTokenizer.from_pretrained('모델이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드함

In [3]:
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


### 2. BERT의 입력
--------------------------
예시 : "Soccer is a really fun [MASK]"

[MASK]의 위치에 해당하는 단어를 예측. 예측 결과를 보기 위해 bert-large-uncased의 토크나이저를 사용하여 해당 문장을 정수 인코딩함

In [4]:
inputs = tokenizer("Soccer is a really fun [MASK].", return_tensors='tf')

토크나이저로 변환된 결과에서 input_ids를 통해 정수 인코딩 결과를 확인할 수 있음

In [5]:
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


토크나이저로 변환된 결과에서 token_type_ids를 통해 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수 있음

In [6]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


입력문장이 한개이므로 문장길이 만큼의 0 시퀀스를 얻음. 만약 문장이 두개였다면 두번째 문장이 시작되는 구간부터는 1의 시퀀스가 나오게 되지만, 여기서는 해당되지 않음

토크나이저로 변환된 결과에서 attention_mask를 통해 실제 단어와 패팅 토큰을 구분하는 용도인 어텐션 마스크를 확인할 수 있음

In [7]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


현재의 입력에서는 패딩이 없으므로 여기서는 문장길이만큼의 1 시퀀스를 얻음. 만약 뒤에 패딩이 있었다면 패딩이 시작되는 구간부터 0의 시퀀스가 나오게 됨


### 3. [MASK] 토큰 예측하기
-------------------------
FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측결과를 정리해서 보여줌. FillMaskPipeline에 우선 앞서 불러온 모델과 토크나이저를 지정해 줌

In [8]:
from transformers import FillMaskPipeline
pipe = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [9]:
pipe('Soccer is a really fun [MASK].')

[{'score': 0.7621126174926758,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.20341917872428894,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208537198603153,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.0018630226841196418,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.0013354861875995994,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [10]:
pipe('The Avengers is a really fun [MASK].')

[{'score': 0.2562899887561798,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.17284108698368073,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107707023620605,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.07248983532190323,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046637684106827,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

In [11]:
pipe("I went to [MASK] this morning.")

[{'score': 0.3573070168495178,
  'token': 2147,
  'token_str': 'work',
  'sequence': 'i went to work this morning.'},
 {'score': 0.23304489254951477,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'i went to bed this morning.'},
 {'score': 0.1284504532814026,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'i went to school this morning.'},
 {'score': 0.06230578571557999,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'i went to sleep this morning.'},
 {'score': 0.046952586621046066,
  'token': 2465,
  'token_str': 'class',
  'sequence': 'i went to class this morning.'}]